# Setup video

In [ ]:
from esper.widget import *
import cv2
import random
import pickle
from esper.table_tennis.utils import *
from esper.table_tennis.video_annotation import *
from esper.table_tennis.pose_utils import *

In [ ]:
video_id = 65
video = Video.objects.filter(id=video_id)[0]
video_ids = [video_id]
video.path

# Find frames with people (Pose)

In [ ]:
LABELED_TAG, _ = Tag.objects.get_or_create(name='openpose:labeled')
frames_with_pose = Frame.objects.filter(video_id=video_id, tags=LABELED_TAG)
num_frame_with_pose = len(frames_with_pose)
print("{} frames contain poses out of {} frames ({:.02f}%)".format(num_frame_with_pose, video.num_frames,
                                                             100. * num_frame_with_pose/video.num_frames))

# Classify sport field Scene
Find long 2 people intervals, build classifier using RGB histogram

## Find frames with only two people

In [ ]:
two_people_intrvlcol_short, two_people_intrvlcol_long = get_two_people_intrvlcol(video_id)

## Create montage 

In [ ]:
match_intervals = intrvlcol2list(match_intrvlcol.get_intervallist(video_id))
create_montage_from_intervals(match_intervals, out_path='/app/result/match_montage.jpg',
                  width=2160, num_cols=10,
                  )

create_video_supercut(match_intervals, '/app/result/match_supercut.mp4')

create_video_montage(match_intervals, '/app/result/match_montage.mp4', 
                     width=2160, num_cols=10)

## Classify match scene

In [ ]:
with open('/app/data/histogram/{:07d}.bin'.format(video_id), 'rb') as f:
    feature_all = np.frombuffer(f.read(), dtype=np.int).reshape((-1, 3*16))

In [ ]:
two_people_intervals_long = intrvlcol2list(two_people_intrvlcol_long, with_duration=True, sort_by_duration=True)

In [ ]:
# not work 
from esper.supercut import manual_select_candidates
manual_select_candidates(interval2result(two_people_intervals_long))

In [ ]:
NUM_SAMPLE_TRAIN = 1000

In [ ]:
# positive samples
selected = [0, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 15, 14, 13, 12, 16, 17, 18, 19, 23, 22, 21, 20, 24, 25, 26, 27, 31, 29, 32, 33, 34, 35, 39, 38, 36, 40, 41, 43, 47, 45, 49, 48]
positive_intervals = [match_intervals[id] for id in selected]
num_sample = 0
POSITIVE_STRIDE = 5
# positive_fids_all = [fid for (vid, sfid, efid, duration) in positive_intervals 
#                  for fid in range(sfid+5, efid-5, POSITIVE_STRIDE)]
positive_fids_all = [fid for (vid, sfid, efid, duration) in positive_intervals 
                 for fid in range(sfid, efid)]
print("Total positive samples ", len(positive_fids_all))
random.shuffle(positive_fids_all)
positive_fids = positive_fids_all[: NUM_SAMPLE_TRAIN]
positive_feature = [feature_all[fid] for fid in positive_fids]
positive_label = [1] * NUM_SAMPLE_TRAIN

In [ ]:
# obsolete
positive_frames = VideoIntervalCollection({video_id: IntervalList([(fid, fid, 0) for fid in positive_fids])})
esper_widget(intrvlcol2result(positive_frames, flat=True))

In [ ]:
create_montage_from_intervals([(video_id, fid, fid) for fid in positive_fids], out_path='/app/result/positive_montage.jpg',
                  width=2160, num_cols=35)

In [ ]:
# negative samples
# NEGATIVE_STRIDE = 20
# video_intrvlcol = VideoIntervalCollection({video_id : IntervalList([(0, video.num_frames-1, 0)])})
# non_match_intrvlcol = video_intrvlcol.minus(two_people_intrvlcol_all)
# negative_intervals = intrvlcol2list(non_match_intrvlcol)
# negative_fids_all = [fid for (vid, sfid, efid, duration) in negative_intervals 
#                     for fid in range(sfid+5, efid-5, NEGATIVE_STRIDE)]
positive_fids_large = [fid for interval in two_people_intrvlcol_all.get_intervallist(video_id).get_intervals() 
                          for fid in range(interval.start, interval.end+1)]
negative_fids_all = [fid for fid in range(0, video.num_frames)
                    if fid not in positive_fids_large]
print("Total negative samples ", len(negative_fids_all))
random.shuffle(negative_fids_all)
negative_fids = negative_fids_all[: NUM_SAMPLE_TRAIN]
negative_feature = [feature_all[fid] for fid in negative_fids]
negative_label = [0] * NUM_SAMPLE_TRAIN

In [ ]:
# obsolete
negative_frames = VideoIntervalCollection({video_id: IntervalList([(fid, fid, 0) for fid in negative_fids])})
esper_widget(intrvlcol2result(negative_frames, flat=True))

In [ ]:
negative_frames = Pose.objects.filter(frame__video_id=video_id, frame__number__in=negative_fids)
len(negative_frames)
# obsolete
esper_widget(qs_to_result(negative_frames, stride=1, limit=1000))

In [ ]:
create_montage_from_intervals([(video_id, fid, fid) for fid in negative_fids], out_path='/app/result/negative_montage.jpg',
                  width=2160, num_cols=35)

In [ ]:
# train model
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=100)

# Train the model using the training sets
model.fit(positive_feature + negative_feature, positive_label + negative_label)

In [ ]:
#Predict Output
predicted_label = model.predict(feature_all) 

In [ ]:
pickle.dump(list(predicted_label), open('/app/data/pkl/match_scene_cls.pkl', 'wb'))

# Segment sport field scene into clip 
Here are two different algorithms

## Filter on two people intervals

In [ ]:
MATCH_THRESH = 0.9
def filter_by_histogram(interval):
    sfid, efid = interval.start, interval.end
    predicted_interval = [predicted_label[fid] for fid in range(sfid, efid)]
    return 1. * np.sum(predicted_interval) / (efid - sfid) > MATCH_THRESH
match_scene_intrvlcol = two_people_intrvlcol.filter(filter_by_histogram)
count_intervals(match_scene_intrvlcol)

In [ ]:
# obsolete
esper_widget(intrvlcol2result(match_scene_intrvlcol, flat=True), use_jupyter_keybindings=True)

In [ ]:
create_montage_from_intervals(intrvlcol2list(match_scene_intrvlcol, with_duration=False), out_path='/app/result/match_scene_montage.jpg',
                  width=2160, num_cols=10)

In [ ]:
pickle.dump(match_scene_intrvlcol, open('/app/result/match_scene_intrvlcol.pkl', 'wb'))

## Merge from classified match scene

In [ ]:
match_scene_cls = pickle.load(open('/app/data/pkl/match_scene_cls.pkl', 'rb'))

In [ ]:
intervals = [Interval(Bounds3D(fid, fid+1)) for fid, cls in enumerate(match_scene_cls) if cls]
match_scene_is = IntervalSet(intervals).dilate(5).coalesce(('t1', 't2'), Bounds3D.span).dilate(-5).filter_size(min_size=25)
match_scene_is.size()

In [ ]:
for idx, intervalSet in enumerate(match_scene_is.get_intervals()):
    print(idx, intervalSet)

In [ ]:
match_scene_ism = IntervalSetMapping({video_id: match_scene_is})

In [ ]:
count_duration(match_scene_ism)

In [ ]:
IntervalSetMapping_to_vgrid(IntervalSetMapping_frame_to_second(match_scene_ism), flat=True)

# Detect Hit

## Process labeled hit
Segment hits into points, label the person and retrive the hit location from openpose 

In [ ]:
match_intervals_all = pickle.load(open('/app/data/pkl/match_scene_intervals_dict.pkl', 'rb'))
match_intervals_A = match_intervals_all['HW_foreground']
match_intervals_B = match_intervals_all['JZ_foreground']
match_intervals_A.sort()
match_intervals_B.sort()
match_ism_A = list_to_IntervalSetMapping(match_intervals_A)
match_ism_B = list_to_IntervalSetMapping(match_intervals_B)

In [ ]:
widget = IntervalSetMapping_to_vgrid(IntervalSetMapping_frame_to_second(match_ism_A), flat=False)
widget

In [ ]:
# manually segment video as two player changes position
foreback_split = {(0, 23): 'HW', (23, 31): 'JZ', (31, 39): 'HW',  (39, 51): 'JZ', (51, 68): 'HW'}

In [ ]:
# collect hits from file name of snapshots
file_list = os.listdir('/app/data/label/final_gold/')
hit_list = []
fps = int(video.fps)
for file_name in file_list:
    time_stamp = file_name.split('.')[2]
    h, m, s, f = time_stamp.split('_')
    fid = int(h) * 60*60*fps + int(m) * 60*fps + int(s) * fps + int(f) + 1 # premier has offset 1
    hit_list.append(fid)
hit_list.sort()
print('Totally %d hits in the video' % len(hit_list))

In [ ]:
# group hits into potins
cur_fid = hit_list[0]
hit_in_point = [[cur_fid]]
for fid in hit_list[1:]:
    if fid - cur_fid < 100:
        hit_in_point[-1].append(fid)
    else:
        hit_in_point.append([fid])
    cur_fid = fid
print('Totally %d points in the video' % len(hit_in_point))

In [ ]:
# for each point, select who gives the first hit
point_ism = IntervalSetMapping({video_id: IntervalSet([Interval(Bounds3D(h[0], h[-1])) for h in hit_in_point])})
widget = IntervalSetMapping_to_vgrid(IntervalSetMapping_frame_to_second(point_ism), flat=True)

In [ ]:
# store who hits the first in each point
# selection = widget.label_state['blocks_selected']
pickle.dump(selection, open('/app/result/label_fg.pkl', 'wb'))

In [ ]:
# propagate whose hit from the first hit of each point
H, W = video.height, video.width
hit_dict = []
for i, point in enumerate(hit_in_point):
    hit_dict.append([])
    for j, fid in enumerate(point):
        pose_fg, pose_bg = get_openpose_by_fid(video_id, fid)
        is_fg = not np.logical_xor(selection[str(i)] == 'Positive', j % 2 == 0)
        pose = pose_fg if is_fg else pose_bg
        if not pose is None:
            kp = pose._format_keypoints()
            pos = kp[Pose.RWrist]
#             if pos.sum() == 0:
#                 pos = (kp[Pose.LShoulder] + kp[Pose.RShoulder]) / 2
            if pos.sum() == 0:
                pos = kp[Pose.Neck]
                assert(pos.sum() != 0)
                print(pos)
            pos = (int(W * pos[0]), int(H * pos[1]))
        else:
            pos = None
        hit_dict[-1].append({'fid': fid, 'fg': is_fg, 'pos': pos})

In [ ]:
# save data
hit_dict_split = {'HW': [], 'JZ': []}
for point in hit_dict:
    m = point[0]['fid'] / video.fps / 60
    for (s, e) in foreback_split.keys():
        if m > s and m < e:
            hit_dict_split[foreback_split[(s, e)]].append(point)
            break
pickle.dump(hit_dict_split, open('/app/data/hit_dict.pkl', 'wb'))
len(hit_dict_split['HW']) + len(hit_dict_split['JZ'])

## Detect sound peak

In [ ]:
match_scene_intrvlcol = pickle.load(open('/app/result/match_scene_intrvlcol.pkl', 'rb'))

In [ ]:
_, sfid, efid, d = intrvlcol2list(match_scene_intrvlcol, sort_by_duration=True)[0]
audio_path = video.extract_audio(segment=(1.*sfid/video.fps, 1.*efid/video.fps))

In [ ]:
# analyze sound
import matplotlib.pyplot as plt
import wave
import sys

spf = wave.open(audio_path,'r')

#Extract Raw Audio from Wav File
signal = spf.readframes(-1)
signal = np.fromstring(signal, 'Int16')
fs = spf.getframerate()
print(fs)

#If Stereo
if spf.getnchannels() == 2:
    print('Just mono files')


Time=np.linspace(0, len(signal)/fs, num=len(signal))

plt.figure(1)
plt.title('Signal Wave...')
plt.plot(Time,signal)
plt.show()

## Detect foot motion

In [ ]:
match_scene_intrvlcol = pickle.load(open('/app/result/match_scene_intrvlcol.pkl', 'rb'))
match_scene_intervals = intrvlcol2list(match_scene_intrvlcol, sort_by_duration=True)

In [ ]:
esper_widget(interval2result(match_scene_intervals[9:10]), disable_caption=True)

In [ ]:
sfid, efid = match_scene_intervals[9][1:3]
# video.download(segment=(sfid/video.fps, efid/video.fps), output_path='/app/tmp/point.mp4')

In [ ]:
joints_to_plot = [Pose.Neck, Pose.LShoulder, Pose.LElbow, Pose.LWrist, Pose.LHip, Pose.LKnee, Pose.LAnkle]
joints_to_plot_name = ['Neck', 'LShoulder', 'LElbow', 'LWrist', 'LHip', 'LKnee', 'LAnkle']
def get_trajectory(fid2pose):
    trajectory_X = {joint: [] for joint in joints_to_plot}
    trajectory_Y = {joint: [] for joint in joints_to_plot}
    xtick_X = {joint: [] for joint in joints_to_plot}
    xtick_Y = {joint: [] for joint in joints_to_plot}
    for fid in sorted(fid2pose):
        keypoints = fid2pose[fid]._format_keypoints()
        for joint in joints_to_plot:
            X = keypoints[joint][0]
            if 0 < X and X < 1: 
                trajectory_X[joint].append(X)
                xtick_X[joint].append(fid)
            Y = keypoints[joint][1]
            if 0 < Y and Y < 1:
                trajectory_Y[joint].append(Y)
                xtick_Y[joint].append(fid)
    return xtick_X, xtick_Y, trajectory_X, trajectory_Y

In [ ]:
plt.figure()
xtick_X, xtick_Y, trajectory_X, trajectory_Y = get_trajectory(foreground_pose)
for id, joint in enumerate(joints_to_plot):
    plt.plot(xtick_X[joint], trajectory_X[joint], label=joints_to_plot_name[id] + 'X')
    plt.plot(xtick_Y[joint], trajectory_Y[joint], '--', label=joints_to_plot_name[id] + 'Y')
legend = plt.legend(loc='lower right')
plt.figure()
xtick_X, xtick_Y, trajectory_X, trajectory_Y = get_trajectory(background_pose)
for id, joint in enumerate(joints_to_plot):
    plt.plot(xtick_X[joint], trajectory_X[joint], label=joints_to_plot_name[id] + 'X')
    plt.plot(xtick_Y[joint], trajectory_Y[joint], '--', label=joints_to_plot_name[id] + 'Y')
legend = plt.legend(loc='lower right')foreground_pose

# Separate foreground/background

In [ ]:
from esper.table_tennis.parse_match import *

In [ ]:
match_scene_intrvlcol = pickle.load(open('/app/result/match_scene_intrvlcol.pkl', 'rb'))
match_scene_intervals = intrvlcol2list(match_scene_intrvlcol, sort_by_duration=True)

In [ ]:
foreground_pose, background_pose = group_pose_from_interval(match_scene_intervals[9])

In [ ]:
hist_foreground_train = collect_histogram(foreground_pose)
hist_background_train = collect_histogram(background_pose)

In [ ]:
# train model
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=100)

# Train the model using the training sets
model.fit(hist_foreground_train + hist_background_train, 
          [1]*len(hist_foreground_train) + [0]*len(hist_background_train))

In [ ]:
match_scene_intrvlcol = pickle.load(open('/app/result/match_scene_intrvlcol.pkl', 'rb'))
match_scene_intervals_foreground = []
match_scene_intervals_background = []
num_sample = 50

for interval in match_scene_intervals:
    foreground_pose, background_pose = group_pose_from_interval(interval)
    if len(foreground_pose) < num_sample:
        continue
    pose_list_sample = random.sample(foreground_pose, num_sample)
    hist_foreground_test = collect_histogram(pose_list_sample)
    
    predicted_label = model.predict(hist_foreground_test)
    if 1. * np.sum(predicted_label) / len(predicted_label) > 0.5:
        match_scene_intervals_foreground.append(interval)
    else:
        match_scene_intervals_background.append(interval)
    print("Foreground", 1. * np.sum(predicted_label) / len(predicted_label))
        

In [ ]:
match_scene_intervals_dict = {'HW_foreground': match_scene_intervals_foreground,
                              'JZ_foreground': match_scene_intervals_background}
pickle.dump(match_scene_intervals_dict, open('/app/result/match_scene_intervals_dict.pkl', 'wb'))

In [ ]:
create_montage(match_scene_intervals_foreground, out_path='/app/result/foreground_montage.jpg',
                  width=2160, num_cols=6)

In [ ]:
create_montage(match_scene_intervals_background, out_path='/app/result/background_montage.jpg',
                  width=2160, num_cols=6)